# Naive Bayes

In [8]:
import pandas as pd
import numpy as np
from sklearn.naive_bayes import CategoricalNB
from sklearn.model_selection import GridSearchCV, StratifiedKFold

#defining the model
nb = CategoricalNB()

In [9]:
#loading in the datasets
df_training = pd.read_csv('../data/archive/train_pca.csv')
df_testing = pd.read_csv('../data/archive/test_pca.csv')

In [10]:
x_train = df_training.iloc[:, :-1]
y_train = df_training.iloc[:,-1]

x_test = df_testing.iloc[:, :-1]
y_test = df_testing.iloc[:,-1]

In [11]:
#Making a grid of values we want our grid search to test to find the best parameters

grid_values = [{'alpha': [.01,.05,.1,1] , 'min_categories': list(range(10,100,10))}]

nb_classifier = GridSearchCV(estimator = nb, param_grid = grid_values, cv = StratifiedKFold(n_splits = 5),
                      scoring = ['accuracy', 'roc_auc_ovr', 'f1_micro'], refit = False, verbose = 0)

nb_model = nb_classifier.fit(x_train, y_train)

C:\Users\lanab\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:610: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\lanab\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 593, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\lanab\anaconda3\lib\site-packages\sklearn\naive_bayes.py", line 1166, in fit
    return super().fit(X, y, sample_weight=sample_weight)
  File "C:\Users\lanab\anaconda3\lib\site-packages\sklearn\naive_bayes.py", line 612, in fit
    X, y = self._check_X_y(X, y)
  File "C:\Users\lanab\anaconda3\lib\site-packages\sklearn\naive_bayes.py", line 1224, in _check_X_y
    check_non_negative(X, "CategoricalNB (input X)")
  File "C:\Users\lanab\anaconda3\lib\site-packages\sklearn\utils\validation.py", line 1068, in check_non_negative
    raise ValueError("Negative v

C:\Users\lanab\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:610: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\lanab\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 593, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\lanab\anaconda3\lib\site-packages\sklearn\naive_bayes.py", line 1166, in fit
    return super().fit(X, y, sample_weight=sample_weight)
  File "C:\Users\lanab\anaconda3\lib\site-packages\sklearn\naive_bayes.py", line 612, in fit
    X, y = self._check_X_y(X, y)
  File "C:\Users\lanab\anaconda3\lib\site-packages\sklearn\naive_bayes.py", line 1224, in _check_X_y
    check_non_negative(X, "CategoricalNB (input X)")
  File "C:\Users\lanab\anaconda3\lib\site-packages\sklearn\utils\validation.py", line 1068, in check_non_negative
    raise ValueError("Negative v

C:\Users\lanab\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:610: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\lanab\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 593, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\lanab\anaconda3\lib\site-packages\sklearn\naive_bayes.py", line 1166, in fit
    return super().fit(X, y, sample_weight=sample_weight)
  File "C:\Users\lanab\anaconda3\lib\site-packages\sklearn\naive_bayes.py", line 612, in fit
    X, y = self._check_X_y(X, y)
  File "C:\Users\lanab\anaconda3\lib\site-packages\sklearn\naive_bayes.py", line 1224, in _check_X_y
    check_non_negative(X, "CategoricalNB (input X)")
  File "C:\Users\lanab\anaconda3\lib\site-packages\sklearn\utils\validation.py", line 1068, in check_non_negative
    raise ValueError("Negative v

C:\Users\lanab\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:610: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\lanab\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 593, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\lanab\anaconda3\lib\site-packages\sklearn\naive_bayes.py", line 1166, in fit
    return super().fit(X, y, sample_weight=sample_weight)
  File "C:\Users\lanab\anaconda3\lib\site-packages\sklearn\naive_bayes.py", line 612, in fit
    X, y = self._check_X_y(X, y)
  File "C:\Users\lanab\anaconda3\lib\site-packages\sklearn\naive_bayes.py", line 1224, in _check_X_y
    check_non_negative(X, "CategoricalNB (input X)")
  File "C:\Users\lanab\anaconda3\lib\site-packages\sklearn\utils\validation.py", line 1068, in check_non_negative
    raise ValueError("Negative v

C:\Users\lanab\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:610: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\lanab\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 593, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\lanab\anaconda3\lib\site-packages\sklearn\naive_bayes.py", line 1166, in fit
    return super().fit(X, y, sample_weight=sample_weight)
  File "C:\Users\lanab\anaconda3\lib\site-packages\sklearn\naive_bayes.py", line 612, in fit
    X, y = self._check_X_y(X, y)
  File "C:\Users\lanab\anaconda3\lib\site-packages\sklearn\naive_bayes.py", line 1224, in _check_X_y
    check_non_negative(X, "CategoricalNB (input X)")
  File "C:\Users\lanab\anaconda3\lib\site-packages\sklearn\utils\validation.py", line 1068, in check_non_negative
    raise ValueError("Negative v

...

C:\Users\lanab\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:610: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\lanab\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 593, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\lanab\anaconda3\lib\site-packages\sklearn\naive_bayes.py", line 1166, in fit
    return super().fit(X, y, sample_weight=sample_weight)
  File "C:\Users\lanab\anaconda3\lib\site-packages\sklearn\naive_bayes.py", line 612, in fit
    X, y = self._check_X_y(X, y)
  File "C:\Users\lanab\anaconda3\lib\site-packages\sklearn\naive_bayes.py", line 1224, in _check_X_y
    check_non_negative(X, "CategoricalNB (input X)")
  File "C:\Users\lanab\anaconda3\lib\site-packages\sklearn\utils\validation.py", line 1068, in check_non_negative
    raise ValueError("Negative v

C:\Users\lanab\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:610: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\lanab\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 593, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\lanab\anaconda3\lib\site-packages\sklearn\naive_bayes.py", line 1166, in fit
    return super().fit(X, y, sample_weight=sample_weight)
  File "C:\Users\lanab\anaconda3\lib\site-packages\sklearn\naive_bayes.py", line 612, in fit
    X, y = self._check_X_y(X, y)
  File "C:\Users\lanab\anaconda3\lib\site-packages\sklearn\naive_bayes.py", line 1224, in _check_X_y
    check_non_negative(X, "CategoricalNB (input X)")
  File "C:\Users\lanab\anaconda3\lib\site-packages\sklearn\utils\validation.py", line 1068, in check_non_negative
    raise ValueError("Negative v

C:\Users\lanab\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:610: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\lanab\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 593, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\lanab\anaconda3\lib\site-packages\sklearn\naive_bayes.py", line 1166, in fit
    return super().fit(X, y, sample_weight=sample_weight)
  File "C:\Users\lanab\anaconda3\lib\site-packages\sklearn\naive_bayes.py", line 612, in fit
    X, y = self._check_X_y(X, y)
  File "C:\Users\lanab\anaconda3\lib\site-packages\sklearn\naive_bayes.py", line 1224, in _check_X_y
    check_non_negative(X, "CategoricalNB (input X)")
  File "C:\Users\lanab\anaconda3\lib\site-packages\sklearn\utils\validation.py", line 1068, in check_non_negative
    raise ValueError("Negative v

C:\Users\lanab\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:610: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\lanab\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 593, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\lanab\anaconda3\lib\site-packages\sklearn\naive_bayes.py", line 1166, in fit
    return super().fit(X, y, sample_weight=sample_weight)
  File "C:\Users\lanab\anaconda3\lib\site-packages\sklearn\naive_bayes.py", line 612, in fit
    X, y = self._check_X_y(X, y)
  File "C:\Users\lanab\anaconda3\lib\site-packages\sklearn\naive_bayes.py", line 1224, in _check_X_y
    check_non_negative(X, "CategoricalNB (input X)")
  File "C:\Users\lanab\anaconda3\lib\site-packages\sklearn\utils\validation.py", line 1068, in check_non_negative
    raise ValueError("Negative v

C:\Users\lanab\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:610: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\lanab\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 593, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\lanab\anaconda3\lib\site-packages\sklearn\naive_bayes.py", line 1166, in fit
    return super().fit(X, y, sample_weight=sample_weight)
  File "C:\Users\lanab\anaconda3\lib\site-packages\sklearn\naive_bayes.py", line 612, in fit
    X, y = self._check_X_y(X, y)
  File "C:\Users\lanab\anaconda3\lib\site-packages\sklearn\naive_bayes.py", line 1224, in _check_X_y
    check_non_negative(X, "CategoricalNB (input X)")
  File "C:\Users\lanab\anaconda3\lib\site-packages\sklearn\utils\validation.py", line 1068, in check_non_negative
    raise ValueError("Negative v

In [13]:
#initializing our lists so we can later store the best params that achieve best scores across metrics
accuracy_best_params = []
roc_auc_best_params = []
f1_best_params = []

accuracy_best_params.append(nb_model.cv_results_['params'][ np.argmin(nb_model.cv_results_['rank_test_accuracy'])])
roc_auc_best_params.append(nb_model.cv_results_['params'][ np.argmin(nb_model.cv_results_['rank_test_roc_auc_ovr'])])
f1_best_params.append(nb_model.cv_results_['params'][ np.argmin(nb_model.cv_results_['rank_test_f1_micro']) ])

In [14]:
accuracy_best_params

[{'alpha': 0.01, 'min_categories': 10}]

In [15]:
roc_auc_best_params

[{'alpha': 0.01, 'min_categories': 10}]

In [16]:
f1_best_params

[{'alpha': 0.01, 'min_categories': 10}]

In [18]:
#finding the accuracy metrics
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score
accuracy_test_score = []

nb_clf = CategoricalNB(alpha= .01, min_categories = 10)
model = nb_clf.fit(x_train, y_train)

y_pred = model.predict(x_test)

accuracy_test_score.append(accuracy_score(y_test, y_pred))

ValueError: Negative values in data passed to CategoricalNB (input X)

In [ ]:
accuracy_test_score